# Exploring multiple approaches to machine learning across multiple small EHD experimental datasets
 - The task is to predict (regress) the size of printed features, as a function of waveform inputs to the EHD printer
 - A second task is to predict (classify) whether a given waveform input will produce any printed pattern at all. (This is equivalent to understanding the print onset voltage threshold.)
 - Multiple hidden confounding variables are likely, such as ink/tip/substrate/atmospheric condition; ink and tip clogging; print tip height from the substrate. Some of these will be relatively constant for each run of experiments, some will vary from feature to feature.
 - Ink dynamics at the printing tip are complex and nonlinear, with electrical and fluid/acoustic phenomena that interact with each other.

## Goals
 - In regression, aim for <3% mean absolute error (from the predicted outcome)
 - In classification, aim for >0.9 ROC AUC
 - For a new/test set, achieve these in <=100 experiments

In [1]:
import sys
import pickle

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from scipy.stats import pearsonr

sys.path.append('..')
from ehd_dataset import EHD_Loader
from ehd_models import EHD_Model


%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
INDEX = "C:/Dropbox/SPEED/Self Driving EHD/Datasets/dataset_index.xlsx"
# INDEX = "C:/Users/Oliver/Dropbox/SPEED/Self Driving EHD/Datasets/dataset_index.xlsx"
loader = EHD_Loader(INDEX)
print("Datasets Loaded!\n>> Quick correlation validation check -- [auac; vec L2] <<")

for i, df in enumerate(loader.datasets):
    AUAC, _ = pearsonr(df.area,
                       df.wave.apply(lambda x: np.sum(np.abs(x))))
    VL2, _ = pearsonr(df.area,
                       df.vector.apply(lambda x: np.sqrt(np.sum(x**2))))
    print(f"<<{loader.names[i]} -- [{AUAC:.3f}; {VL2:.3f}]", end='>> ')

Failed to load 10-Mar-2022 large nozzle mosaic: 'DataFrame' object has no attribute 'note'
dataset C:\Dropbox\SPEED\Self Driving EHD\Datasets\29-Mar-2022 lg 1cm 300 points	263 points	offset 2	corr 0.4979414348873561
dataset C:\Dropbox\SPEED\Self Driving EHD\Datasets\2-May-2022__run 1	121 points	offset 32	corr 0.6417618745477631
dataset C:\Dropbox\SPEED\Self Driving EHD\Datasets\2-May-2022__run 2	528 points	offset 4	corr 0.6184015138069094
dataset C:\Dropbox\SPEED\Self Driving EHD\Datasets\23-May-2022_squares	352 points	offset 8	corr 0.5521885646579606
dataset C:\Dropbox\SPEED\Self Driving EHD\Datasets\24-May-2022 large harmonics	682 points	offset 0	corr 0.5871179189117662
dataset C:\Dropbox\SPEED\Self Driving EHD\Datasets\8-Aug-2022_lg-square-20um	686 points	offset 1	corr 0.8454268563076222
dataset C:\Dropbox\SPEED\Self Driving EHD\Datasets\10-Sep-2022_std-square-10um	149 points	offset 16	corr 0.7935638727333442
dataset C:\Dropbox\SPEED\Self Driving EHD\Datasets\10-Sep-2022_std-square-

In [4]:
df = loader.datasets[0]
np.isnan(df['clogging'].loc[0])

False

In [5]:
# REGRESSION MODELS
from ehd_models import EHD_Model

# Print Area Regression Models <<<<<<<<<<<<<<<<<<<<
REGRESSION_model_architectures = {'MLE': {},
                                  'cold_RF': {},
                                  'only_pretrained_RF': {},
                                  # 'cold_MLP': {'max_iter': 100_000},
                                  # 'lastXY_MLP': {'max_iter': 200_000},
                                  # 'lastXY_RF': {}
                                 }
NEW_CACHE = False
OVERWRITE_RESULTS = True

XTYPE = "vector"
YTYPE = "max_width"
FILTERS = [
           ('vector',  lambda x: len(x), 2),
           ('Wavegen', lambda x: x, 'square'),
           ('V Thresh [V] @ .5s', np.isnan, False),
           ('jetted',  lambda x: x, True),
           ('clogging', lambda x: x, False)
          ]

CACHE_NAME = 'regression.cache'

if NEW_CACHE:
    reg_results = []
    reg_names = []
else:
    with open(CACHE_NAME, 'rb') as f:
        (reg_results, reg_names) = pickle.load(f)

for architecture, params in REGRESSION_model_architectures.items():
    if (architecture in reg_names) and not OVERWRITE_RESULTS:
        print(f'{architecture} was already evaluated - loaded from cache')
    else:
        print('')
        if architecture in reg_names:
            print(f"Overwriting previous results from {architecture}")
            reg_results = [reg_results[i] for i in range(len(reg_results)) if reg_names[i] != architecture]
            reg_names = [r for r in reg_names if r != architecture]
        print(f'Evaluating regression model type: {architecture}')
        model = EHD_Model(architecture=architecture, params=params)

        # for dataset_fold in range(loader.num_folds()):
        for dataset_fold in range(loader.num_folds(FILTERS)):
            print(f"fold {dataset_fold}", end=" ")
            pretrain_set, eval_set, eval_name = loader.folded_dataset(fold=dataset_fold, xtype=XTYPE, ytype=YTYPE,
                                                                     pretrain=model.pretrainer, filters=FILTERS)
            # try:
            if model.pretrainer:
                print('pretrain...', end=' ')
                model.pretrain(pretrain_set)
                print('done', end=' ')

            output = model.evaluate(eval_set)

            output['architecture'] = architecture
            output['eval_dataset'] = eval_name
            reg_names.append(architecture)
            reg_results.append(output)

with open(CACHE_NAME, 'wb') as f:
    pickle.dump((reg_results, reg_names), f)

Overwriting previous results from MLE

Evaluating regression model type: MLE
fold 0 pretrain... done fold 1 pretrain... done fold 2 pretrain... done fold 3 pretrain... done Overwriting previous results from cold_RF

Evaluating regression model type: cold_RF
fold 0 fold 1 fold 2 fold 3 Overwriting previous results from only_pretrained_RF

Evaluating regression model type: only_pretrained_RF
fold 0 pretrain... done fold 1 pretrain... done fold 2 pretrain... done fold 3 pretrain... done 

In [7]:
# JETTING CLASSIFICATION MODELS
from ehd_models import EHD_Model

# Jetting Classification Models <<<<<<<<<<<<<<<<<<<<
CLASS_model_architectures = {'MLE_class': {},
                             'cold_RF_class': {},
                             'only_pretrained_RF_class': {},
                             # 'cold_MLP_class': {'max_iter': 100_000},
                             # 'lastXY_MLP_class': {'max_iter': 200_000},
                             # 'lastXY_RF_class': {}
                            }
NEW_CACHE = False
OVERWRITE_RESULTS = True

XTYPE = "vector"
YTYPE = "jetted"
FILTERS = [
           ('vector', lambda x: len(x), 2),
           ('Wavegen', lambda x: x, 'square'),
           ('V Thresh [V] @ .5s', np.isnan, False),
           ('clogging', lambda x: x, False)
          ]

CACHE_NAME = 'classification.cache'

if NEW_CACHE:
    class_results = []
    class_names = []
else:
    with open(CACHE_NAME, 'rb') as f:
        (class_results, class_names) = pickle.load(f)
    
for architecture, params in CLASS_model_architectures.items():
    if (architecture in class_names) and not OVERWRITE_RESULTS:
        print(f'{architecture} was already evaluated - loaded from cache')
    else:
        if architecture in class_names:
            print(f"Overwriting previous results from {architecture}")
            class_results = [class_results[i] for i in range(len(class_results)) if class_names[i] != architecture]
            class_names = [c for c in class_names if c != architecture]
        print(f'\nEvaluating classification model type: {architecture}')
        model = EHD_Model(architecture=architecture, params=params)

        for dataset_fold in range(loader.num_folds(FILTERS)):
            print(f"fold {dataset_fold}", end=" ")
            pretrain_set, eval_set, eval_name = loader.folded_dataset(fold=dataset_fold, xtype=XTYPE, ytype=YTYPE,
                                                                     pretrain=model.pretrainer, filters=FILTERS)
            model.pretrain(pretrain_set)
            output = model.evaluate(eval_set)

            output['architecture'] = architecture
            output['eval_dataset'] = eval_name
            class_names.append(architecture)
            class_results.append(output)

with open(CACHE_NAME, 'wb') as f:
    pickle.dump((class_results, class_names), f)

Overwriting previous results from MLE_class

Evaluating classification model type: MLE_class
fold 0 fold 1 fold 2 fold 3 Overwriting previous results from cold_RF_class

Evaluating classification model type: cold_RF_class
fold 0 fold 1 fold 2 fold 3 Overwriting previous results from only_pretrained_RF_class

Evaluating classification model type: only_pretrained_RF_class
fold 0 fold 1 fold 2 fold 3 

In [9]:
# CREATE FIG and SUMMARY FILES
import seaborn as sns

OUT_XLSX = 'eval_summary.xlsx'
xlsx_writer = pd.ExcelWriter(OUT_XLSX)
plt.rcParams['figure.figsize'] = (11.0, 10.0) # set default size of plots
plt.clf()

def n_walk_result_vis(df, x, y, trends, ax=None, outfile=None, legend='auto', labels=None):
    bar_ci = 0.95  # 'sd' for standard deviation
    sns.lineplot(
        data=df.loc[~df['unique N']],
        x=x, y=y, hue=trends, style=trends,
        markers=True,
        err_style="bars", ci=bar_ci,
        markersize=7,
        linewidth=1.8,
        ax=ax,
        legend=legend
    )
    ticks = df[x].loc[~df['unique N']].unique()
    ax.set_xticks(ticks=ticks,
               labels=np.round(10**ticks).astype(int))
    ax.set_xlabel('dataset size')
    if legend is not None and labels is not None:
        ax.legend(labels)
    if y == 'MSE':
        ax.set_ylim(0, 16e3)
    if y == 'MAE':
        ax.set_ylim(0, 40)
    if y == 'r':
        ax.set_ylim(0, 1)
    if y == 'F1':
        ax.set_ylim(0.49, 1)
    if y == 'AUC':
        ax.set_ylim(0.49, 1)
    if y == 'MAPE':
        ax.set_ylim(0, 100)
    if outfile is not None:
        plt.savefig(outfile, bbox_inches='tight', pad_inches=0.1, transparent=False, dpi=500)
        plt.clf()

# REGRESSION: Expand, log, and visualize results
df = pd.concat(reg_results, ignore_index=True)
df['log_size'] = df['train_size'].apply(lambda x: np.log10(x).round(3))
df['MAPE'] = df['MAPE'].apply(lambda x: 100 * x)
df['unique N'] = df['train_size'].apply(lambda x: (df['train_size'] == x).sum() <= len(df['architecture'].unique()))

fig, ax = plt.subplots(2,2)

legend = 'auto'
# labels = ('MLE', 'MLP: last X/Y', 'Rand. Forest: pretrain only', 'Rand. Forest: last X/Y', 'Rand. Forest: cold start')
labels = ('MLE', 'Cold Random Forest', 'Pretrained Random Forest')
for i, metric in enumerate(['MAE', 'r']):  # MSLE and MAPE left out for now + 'MSE', 
    #n_walk_result_vis(df, x='log_size', y=metric, trends='architecture', outfile=f"{metric}.png")
    n_walk_result_vis(df, x='log_size', y=metric, trends='architecture', ax=ax[0][i], legend=legend, labels=labels)
    labels = None
    legend = False
    
df.to_excel(xlsx_writer, index=False, sheet_name='Regression')

# CLASSIFICATION: Expand, log, and visualize results
df = pd.concat(class_results, ignore_index=True)
df['log_size'] = df['train_size'].apply(lambda x: np.log10(x).round(3))
df['unique N'] = df['train_size'].apply(lambda x: (df['train_size'] == x).sum() <= len(df['architecture'].unique()))
    
df.to_excel(xlsx_writer, index=False, sheet_name='Classification')

legend = 'auto'
labels = ('MLE', 'Cold Random Forest', 'Pretrained Random Forest')
# labels = ('MLE', 'MLP: last X/Y', 'Rand. Forest: pretrain only', 'Rand. Forest: last X/Y', 'Rand. Forest: cold start', 'MLP: cold start')
for i, metric in enumerate(['AUC', 'F1']):
    #n_walk_result_vis(df, x='log_size', y=metric, trends='architecture', outfile=f"{metric}.png")
    n_walk_result_vis(df, x='log_size', y=metric, trends='architecture', ax=ax[1][i], legend=legend, labels=labels)
    labels = None
    legend = False


plt.savefig('4-square_metrics.png', bbox_inches='tight', pad_inches=0.1, transparent=False, dpi=300)
plt.clf()
xlsx_writer.save()
xlsx_writer.close()

<Figure size 792x720 with 0 Axes>

<Figure size 792x720 with 0 Axes>